# Joinmarket Fees Analysis
Loading and cleaning the data

In [ ]:
# Fee Analysis Notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import data handling functions
from preprocessing.utils import get_snapshot_filepaths
from preprocessing.dataframe import load_dataframe, save_dataframe

In [ ]:
df_stats_uncleaned = load_dataframe('../dataframe.pkl')

In [ ]:
# Extreme fees, not valid for computation
column = 'relative_fees_percentage_mean'
extreme_records = df_stats_uncleaned[df_stats_uncleaned[column] > 1]
print("Extreme records:")
display(extreme_records[[column, 'relative_fees_satoshis_mean']])


In [ ]:
df_stats = df_stats_uncleaned[df_stats_uncleaned['relative_fees_percentage_mean'] <= 1]

In [ ]:
# Choose the fee column of interest. For example:
fee_col = 'relative_fees_percentage_mean'

# Calculate the 99th percentile threshold
threshold = df_stats[fee_col].quantile(0.99)
print(f"Top 1% threshold for {fee_col}: {threshold}")

# Filter the DataFrame to only include records at or above the threshold and sort in descending order
top_1_percent = df_stats[df_stats[fee_col] >= threshold].sort_values(by=fee_col, ascending=False)

# Display the top 1% records sorted by the fee column
display(top_1_percent)


In [ ]:
df_stats.head()

In [ ]:
df_stats

### Overall Fee Distribution
The data is collected in 5 minute snapshots. Metrics are computed for each snapshot and stored into the dataframe we are working with.
Bellow is computed the distribution of those metrics within the snapshots.
The metrics we are working with are:
- Relative Fees Percentage Mean 
- Relative Fees Satoshis Mean (computed from the value of the offer)
- Relative Fees Satoshis Median
- Absolute Fees Satoshis Means (For absorder, the opposite of relorder, not total)
- Absolute Fees Satoshis Median
- Ration of Relative to Absolute Fees (Percentage of rel. fees of the total)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Relative fees (mean)
min_bin = 0
max_bin = 0.005
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

sns.histplot(data=df_stats, x='relative_fees_percentage_mean', bins=bins, ax=axes[0, 0])
axes[0, 0].set_xlim([min_bin, max_bin])
axes[0, 0].set_title('Relative Fees (Mean)')
axes[0, 0].set_xlabel('Fee Percentage')

# Plot 2: Absolute fees (mean)
min_bin = 0
max_bin = 15000
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

sns.histplot(data=df_stats, x='absolute_fees_satoshis_mean', bins=bins, ax=axes[0, 1])
axes[0, 1].set_xlim([min_bin, max_bin])
axes[0, 1].set_title('Absolute Fees (Mean)')
axes[0, 1].set_xlabel('Satoshis')

# Plot 3: Relative fees (satoshis mean)
min_bin = 0
max_bin = 2000  # adjust as needed
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

sns.histplot(data=df_stats, x='relative_fees_satoshis_mean', bins=bins, ax=axes[0, 2])
axes[0, 2].set_xlim([min_bin, max_bin])
axes[0, 2].set_title('Relative Fees (Satoshis Mean)')
axes[0, 2].set_xlabel('Satoshis')

# Plot 4: Relative fees (satoshis median)
min_bin = 0
max_bin = 1000  # adjust based on your data
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

sns.histplot(data=df_stats, x='relative_fees_satoshis_median', bins=bins, ax=axes[1, 0])
axes[1, 0].set_xlim([min_bin, max_bin])
axes[1, 0].set_title('Relative Fees (Satoshis Median)')
axes[1, 0].set_xlabel('Satoshis')

# Plot 5: Absolute fees (satoshis median)
min_bin = 0
max_bin = 15000  # same as above for consistency
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

sns.histplot(data=df_stats, x='absolute_fees_satoshis_median', bins=bins, ax=axes[1, 1])
axes[1, 1].set_xlim([min_bin, max_bin])
axes[1, 1].set_title('Absolute Fees (Satoshis Median)')
axes[1, 1].set_xlabel('Satoshis')

# Hide empty sixth subplot
min_bin = 0
max_bin = 1
bin_width = (max_bin - min_bin) / 50
bins = np.arange(min_bin, max_bin + bin_width, bin_width)

plt.figure(figsize=(10, 6))
sns.histplot(data=df_stats, x='relative_fees_ratio', bins=bins, ax=axes[1, 2])
axes[1, 2].set_title('Ratio of Relative vs Absolute Fees by Count')
axes[1, 2].set_xlabel('Relative Fees Ratio')
axes[1, 2].set_ylabel('Count')
axes[1, 2].set_xlim([min_bin, max_bin])


plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# Define original bin specs (for reference) for each attribute
# (min_spec, max_spec, number_of_bins)
bin_specs = {
    'relative_fees_percentage_mean': (0, 0.005, 50),
    'absolute_fees_satoshis_mean': (0, 15000, 50),
    'relative_fees_satoshis_mean': (0, 2000, 50),
    'relative_fees_satoshis_median': (0, 1000, 50),
    'absolute_fees_satoshis_median': (0, 15000, 50),
    'relative_fees_ratio': (0, 1, 50),
}

# Compute histogram data using each column's actual min and max values
histogram_data = {}
for column, (spec_min, spec_max, num_bins) in bin_specs.items():
    data = df_stats[column].dropna()
    # Use the actual minimum and maximum of the data
    full_min = data.min()
    full_max = data.max()
    bins = np.linspace(full_min, full_max, num_bins + 1)
    counts, bin_edges = np.histogram(data, bins=bins)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    histogram_data[column] = pd.DataFrame({
        'Bin Start': bin_edges[:-1],
        'Bin End': bin_edges[1:],
        'Bin Center': bin_centers,
        'Count': counts
    })

In [ ]:
display_histogram_data = False
if display_histogram_data:
    histogram_data

### Overall statistics
Computed the weighted means of the metrics we are collecting per snapshot over all snapshots.
Additionally is computed the ocombined mean of all fees.

In [ ]:
import pandas as pd
import numpy as np

def weighted_mean(df, value_col, weight_col):
    valid = df[value_col].notna() & df[weight_col].notna()
    weighted_sum = (df.loc[valid, value_col] * df.loc[valid, weight_col]).sum()
    total_weight = df.loc[valid, weight_col].sum()
    return weighted_sum / total_weight if total_weight > 0 else np.nan

def compute_weighted_summary(df):
    # Compute correct absolute fee count
    df = df.copy()
    df['absolute_fees_count'] = df['all_fees_count'] - df['relative_fees_count']

    df['all_fees_satoshis_mean'] = (
        df['absolute_fees_satoshis_mean'] * df['absolute_fees_count'] +
        df['relative_fees_satoshis_mean'] * df['relative_fees_count']
    ) / df['all_fees_count']
    result = {}

    specs = [
        ('relative_fees_percentage_mean', 'relative_fees_count', 'Relative Fees (Percentage)'),
        ('absolute_fees_satoshis_mean', 'absolute_fees_count', 'Absolute Fees (sats)'),
        ('relative_fees_satoshis_mean', 'relative_fees_count', 'Relative Fees (sats, mean)'),
        ('relative_fees_satoshis_median', 'relative_fees_count', 'Relative Fees (sats, median)'),
        ('absolute_fees_satoshis_median', 'absolute_fees_count', 'Absolute Fees (sats, median)'),
        ('all_fees_satoshis_mean', 'all_fees_count', 'All Fees (sats)'),
        ('relative_fees_ratio', 'relative_fees_count', 'Relative to Absolute Fees Ratio')
    ]

    for value_col, weight_col, label in specs:
        wm = weighted_mean(df, value_col, weight_col)
        desc = df[value_col].describe()

        result[label] = {
            'Weighted': wm,
            'Std Dev': desc['std'],
            'Min': desc['min'],
            'Max': desc['max'],
            'Valid Snapshots': int(df[value_col].notna().sum()),
        }

    # Format output nicely
    result_df = pd.DataFrame(result).T
    pd.options.display.float_format = '{:,.1f}'.format
    return result_df.round({
        'Weighted Mean': 8,
        'Unweighted Mean': 8,
        'Std Dev': 8,
        'Min': 8,
        'Max': 8
    })

# Compute the summary DataFrame as usual
summary_df = compute_weighted_summary(df_stats)

# Create a Styler with the default 8-decimal format for all values
styled = summary_df.style.format("{:,.1f}")
styled = styled.format("{:,.5f}", subset=pd.IndexSlice["Relative Fees (Percentage)", :])
styled = styled.format("{:,.5f}", subset=pd.IndexSlice["Relative to Absolute Fees Ratio", :])

display(styled)

# Monthly statistics
Same data as above grouped monthly

In [ ]:
import pandas as pd
import numpy as np

def weighted_mean(df, value_col, weight_col):
    valid = df[value_col].notna() & df[weight_col].notna()
    weighted_sum = (df.loc[valid, value_col] * df.loc[valid, weight_col]).sum()
    total_weight = df.loc[valid, weight_col].sum()
    return weighted_sum / total_weight if total_weight > 0 else np.nan

def compute_weighted_summary(df):
    # Compute correct absolute fee count from cumulative counts.
    df = df.copy()
    df['absolute_fees_count'] = df['all_fees_count'] - df['relative_fees_count']

        # Compute all fees mean (in satoshis) as the weighted average of absolute and relative fees.
    df['all_fees_satoshis_mean'] = (
        df['absolute_fees_satoshis_mean'] * df['absolute_fees_count'] +
        df['relative_fees_satoshis_mean'] * df['relative_fees_count']
    ) / df['all_fees_count']
    
    result = {}
    specs = [
        ('relative_fees_percentage_mean', 'relative_fees_count', 'Relative Fees (Percentage)'),
        ('absolute_fees_satoshis_mean', 'absolute_fees_count', 'Absolute Fees (sats)'),
        ('relative_fees_satoshis_mean', 'relative_fees_count', 'Relative Fees (sats, mean)'),
        ('relative_fees_satoshis_median', 'relative_fees_count', 'Relative Fees (sats, median)'),
        ('absolute_fees_satoshis_median', 'absolute_fees_count', 'Absolute Fees (sats, median)'),
        ('all_fees_satoshis_mean',        'all_fees_count', 'All Fees (sats)'),
        ('all_fees_satoshis_mean', 'all_fees_count', 'All Fees (sats)'),
        ('relative_fees_ratio', 'relative_fees_count', 'Relative to Absolute Fees Ratio')
    ]
    
    for value_col, weight_col, label in specs:
        wm = weighted_mean(df, value_col, weight_col)
        desc = df[value_col].describe()
        result[label] = {
            'Weighted': wm,
            'Std Dev': desc['std'],
            'Min': desc['min'],
            'Max': desc['max'],
            'Valid Snapshots': int(df[value_col].notna().sum()),
        }
    result_df = pd.DataFrame(result).T
    return result_df

def compute_monthly_weighted_summary(df, timestamp_col='timestamp'):
    df = df.copy()
    # Check if the timestamp column exists; if not, check if the index is a DatetimeIndex.
    if timestamp_col not in df.columns:
        if isinstance(df.index, pd.DatetimeIndex):
            df[timestamp_col] = df.index
        else:
            datetime_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
            if not datetime_cols:
                raise KeyError("No datetime column found in DataFrame.")
            timestamp_col = datetime_cols[0]
            print(f"Using '{timestamp_col}' as the timestamp column.")
    # Ensure the timestamp column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Create a 'month' column based on the timestamp
    df['month'] = df[timestamp_col].dt.to_period('M').dt.to_timestamp()
    
    monthly_summaries = []
    for month, group in df.groupby('month'):
        summary = compute_weighted_summary(group)
        summary['Month'] = month
        summary['Metric'] = summary.index
        monthly_summaries.append(summary.reset_index(drop=True))
    monthly_summary_df = pd.concat(monthly_summaries, ignore_index=True)
    return monthly_summary_df

# Compute monthly weighted summary (using the 'timestamp' column, whether it's in the index or not)
monthly_summary_df = compute_monthly_weighted_summary(df_stats, timestamp_col='timestamp')

# Extract only the relevant columns: 'Metric', 'Month', and 'Weighted'
weighted_only = monthly_summary_df[['Metric', 'Month', 'Weighted']]

# Pivot the DataFrame so that rows are metrics and columns are months.
# Each cell contains the Weighted value.
pivoted = weighted_only.pivot(index='Metric', columns='Month', values='Weighted')
pivoted = pivoted.sort_index(axis=1)  # Sort columns by month

# Custom Order
custom_order = [
    'Relative Fees (Percentage)',
    'Absolute Fees (sats)',
    'Relative Fees (sats, mean)',
    'Relative Fees (sats, median)',
    'Absolute Fees (sats, median)',
    'All Fees (sats)',
    'Relative to Absolute Fees Ratio'
]

pivoted = pivoted.reindex(custom_order)

# Create a Styler with the default 8-decimal format for all values
styled = pivoted.style.format("{:,.1f}")
styled = styled.format("{:,.5f}", subset=pd.IndexSlice["Relative Fees (Percentage)", :])
styled = styled.format("{:,.5f}", subset=pd.IndexSlice["Relative to Absolute Fees Ratio", :])



display(styled)


# Timeline

Rolling mean of the metrics.
You can set the size of the rolling windows in days.

In [ ]:
rolling_window_days = 2
window_size = 12 * 24 * rolling_window_days

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure that 'timestamp' exists as a column; if not, use the index.
if 'timestamp' not in df_stats.columns:
    df_stats = df_stats.copy()  # avoid modifying original
    df_stats['timestamp'] = df_stats.index
df_stats['timestamp'] = pd.to_datetime(df_stats['timestamp'])

# Define the metrics you want to plot.
metrics = [
    'relative_fees_percentage_mean',
    'absolute_fees_satoshis_mean',
    'relative_fees_satoshis_mean',
    'relative_fees_satoshis_median',
    'absolute_fees_satoshis_median',
    'relative_fees_ratio'
]

# Create a new DataFrame with rolling averages for each metric.
df_rolling = df_stats.copy()
for metric in metrics:
    df_rolling[metric + '_rolling'] = df_rolling[metric].rolling(window=window_size, min_periods=1).mean()

# Create subplots to plot the rolling average time series for each metric.
fig, axes = plt.subplots(nrows=len(metrics), ncols=1, figsize=(12, 2 * len(metrics)), sharex=True)
if len(metrics) == 1:
    axes = [axes]

for ax, metric in zip(axes, metrics):
    ax.plot(df_rolling['timestamp'], df_rolling[metric + '_rolling'], linestyle='-', label=f"{metric} (rolling avg)")
    ax.set_ylabel(metric)
    ax.legend(loc='upper right')

axes[-1].set_xlabel('Timestamp')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
